# Table of Contents

- [Imports and Configuration](#Imports-and-Configuration)
- [Mapping](#Mapping)

# Imports and Configuration

Import necessary packages and configure APIs.
Here we are using google directions API to regenerate the route that we will map using folium
Folium is used to create an interactive map

In [1]:
import folium
import numpy as np
import pandas as pd
import geocoder
import googlemaps
from folium import IFrame
import base64
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# get Google API key to plot route
with open('../api_key_maps.txt') as f:
    api_key = f.readline()
    f.close    

# Load dataframes with stopover city names, and information about photos 
df = pd.read_csv('../data/datasets/photos_with_pred.csv')
route_df = pd.read_csv('../data/datasets/route.csv')
route_df.drop(columns=['Unnamed: 0'], inplace=True)

In [3]:
# List with city names
cities = []
for i in range(route_df.shape[0]):
    cities.append(route_df.iloc[i]['Cities'])

# List with city center coordinates
def get_latlong(address):
    g = geocoder.arcgis(address)
    return tuple(g.latlng)

# Get the latitude and longitude of each city
cities_coord = [get_latlong(city) for city in cities]

# Dict with bounding boxes of each city
bounding_box = {a: {} for a in cities}
for index, city in enumerate(cities):
    g = geocoder.arcgis(cities[index])
    bounding_box[city]['max_lat'] = g.bbox['northeast'][0]
    bounding_box[city]['min_lat'] = g.bbox['southwest'][0]
    bounding_box[city]['max_lng'] = g.bbox['northeast'][1]
    bounding_box[city]['min_lng'] = g.bbox['southwest'][1]

# Get the highest and lowest coordinates, to know where to center the map
map_minlat = min([x['min_lat'] for x in bounding_box.values()])
map_maxlat = max([x['max_lat'] for x in bounding_box.values()])
map_minlng = min([x['min_lng'] for x in bounding_box.values()])
map_maxlng = max([x['max_lng'] for x in bounding_box.values()])

# map center
avg_lat = (map_minlat + map_maxlat) / 2
avg_lng = (map_minlng + map_maxlng) / 2

# Mapping

The goal is to generate a map where we can see the route and the locations of the pictures of interest. Circles will be of different colors depending on which class they belong to
Then the user can zoom in on the map see where the most interesting neighborhoods are, and click on a marker to view the pictures

In [4]:
# Re-generate the map
client = googlemaps.client.Client(key = api_key)
route = googlemaps.directions.directions(client = client,
                                         origin = route_df.iloc[0]['Cities'],
                                         destination = route_df.iloc[route_df.shape[0]-1]['Cities'], 
                                         mode = 'driving')

# Since we can't plot routes with folium we plot lines that join coordinates of the begining and end of every
# step in the route
all_stops = []
for step in route[0]['legs'][0]['steps']:
    all_stops.append([(step['start_location']['lat'], step['start_location']['lng']), 
                      (step['end_location']['lat'], step['end_location']['lng'])])

In [5]:
# popup help from https://github.com/python-visualization/folium/issues/604

# Instantiate Map object centered at the midpoint between cities
mapit = folium.Map(location = [avg_lat, avg_lng], zoom_start=6)

# plot a line for every step of the route
for index in range(len(all_stops)):
    folium.PolyLine(all_stops[index]).add_to(mapit)

# take the subset of the dataframe that was successfuly classified (with probability > 0.5)
df = df[df['labels'] != '']   

# step is 14 otherwise the map won't show, so the map only had 1/14th of our total data
for i in range(0, df.shape[0], 14):
    marker_lat = df.iloc[i]['latitude']
    marker_lng = df.iloc[i]['longitude']
    label = df.iloc[i]['labels']
    url = df.iloc[i]['url']
    
    # get the images to pop up when we click on the circle
    html = '<img src="{}">'.format(url)
    resolution, width, height = 30, 20, 10
    iframe = IFrame(html, width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe)
    
    # color differently according to labels
    if label == 'food':
        circle_color = 'crimson'
    elif label == 'park':
        circle_color = 'green'
    elif label == 'view':
        circle_color = 'blue'
    elif label == 'culture':
        circle_color = 'orange'
    
    # add circle to the map
    folium.Circle(location = [marker_lat, marker_lng], 
                  color = circle_color,
                  fill_color=circle_color,
                  radius = 100,
                  popup = popup,
                  fill = True).add_to(mapit)
mapit

In [8]:
# save html file
mapit.save('../output/mapit.html')